In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import numpy as np
from scipy import stats
import seaborn as sns
from functools import reduce
pd.options.mode.chained_assignment = None 
from statistics import mean
from profile_generation import *
import warnings
warnings.simplefilter('ignore', np.RankWarning)
from scipy.optimize import curve_fit
import matplotlib.patches as patches
from scipy.optimize import curve_fit
from openpyxl import load_workbook
pd.set_option('display.max_rows', None)

In [22]:
input_file = 'Retrospective Liver Transplant Data.xlsx'
rows_to_skip = 17

# Get list of patients/sheet names
list_of_patients = get_sheet_names(input_file)

# Define lists
list_of_patient_df = []
list_of_cal_pred_df = []
list_of_result_df = []

patients_to_exclude_linear = []
patients_to_exclude_quad = []

for patient in list_of_patients:
        
    df = pd.read_excel(input_file, sheet_name=patient, skiprows=rows_to_skip)
    
    df = clean_data(df, patient)
    df = keep_ideal_data(df, patient, list_of_patient_df)
        
    # Choose and keep data for calibration and efficacy-driven dosing
    cal_pred_linear, patients_to_exclude_linear = cal_pred_data(df, patient, patients_to_exclude_linear, 1)
    cal_pred_quad, patients_to_exclude_quad = cal_pred_data(df, patient, patients_to_exclude_quad, 2)
    
    # Keep patient data with sufficient dose-response pairs and predictions
    cal_pred, list_of_cal_pred_df = keep_target_patients(patient, patients_to_exclude_linear, patients_to_exclude_quad, 
                                                     cal_pred_linear, cal_pred_quad, list_of_cal_pred_df)

    # Prepare dataframe for prediction
    # Create result DataFrame
#     max_count_input = len(cal_pred[cal_pred['type'] =='linear'])
#     print(f"{patient}: max_count_input {max_count_input} | {cal_pred}")
#     col_names = ['patient', 'method', 'pred_day'] + \
#                 ['fit_dose_' + str(i) for i in range(1, max_count_input + 1)] + \
#                 ['fit_response_' + str(i) for i in range(1, max_count_input + 1)] + \
#                 ['dose', 'response', 'prev_coeff_2x', 'prev_coeff_1x', 'prev_coeff_0x',\
#                  'prev_deviation', 'coeff_2x', 'coeff_1x', 'coeff_0x', 'prediction', 'deviation',
#                  'abs_deviation']
#     result = pd.DataFrame(columns=col_names)
    
#     j = 0
#     if patient not in patients_to_exclude_linear:
#         deg = 1
        
#         # Prepare dataframe for L_Cum_wo_origin
#         for i in range(deg + 1, len(cal_pred[cal_pred['type']=='linear'])):
#             result.loc[j, 'patient'] = cal_pred.loc[i, 'patient']
#             result.loc[j, 'method'] = 'L_Cum_wo_origin'
#             result.loc[j, 'pred_day'] = cal_pred.loc[i, 'day']
#             # result.loc[j, 'fit_dose_1':'fit_dose_' + str(i + deg + 1)] = cal_pred.loc[0:i-1, 'dose']
#             result.loc[j, 'fit_dose_1':'fit_dose_' + str(i)] = cal_pred.loc[0:i-1, 'dose'].to_numpy()
#             result.loc[j, 'fit_response_1':'fit_response_' + str(i)] = cal_pred.loc[0:i-1, 'response'].to_numpy()
#             result.loc[j, 'dose'] = cal_pred.loc[i, 'dose']
#             result.loc[j, 'response'] = cal_pred.loc[i, 'response']
#             j = j + 1
            
        # list_of_result_df.append(result)
            
        # print(result)


# Print patients to exclude        
patients_to_exclude_linear = sorted(set(patients_to_exclude_linear))
patients_to_exclude_quad = sorted(set(patients_to_exclude_quad))
print(f"Patients to exclude for linear methods: {patients_to_exclude_linear}")
print(f"Patients to exclude for quad methods: {patients_to_exclude_quad}")

# Join dataframes from individual patients
df = pd.concat(list_of_patient_df)
df.reset_index(inplace=True, drop=True)
cal_pred = pd.concat(list_of_cal_pred_df)
# result_df = pd.concat(list_of_result_df)

# result_df.columns


    day response dose patient       type
0   2.0      2.4  0.5      84     linear
1   3.0      2.8  1.0      84     linear
2   4.0      3.2  1.5      84     linear
3   5.0      3.1  1.5      84     linear
4   6.0      7.9  3.0      84     linear
5   7.0       10  3.0      84     linear
6   8.0     10.3  3.0      84     linear
7   2.0      2.4  0.5      84  quadratic
8   3.0      2.8  1.0      84  quadratic
9   4.0      3.2  1.5      84  quadratic
10  5.0      3.1  1.5      84  quadratic
11  6.0      7.9  3.0      84  quadratic
12  7.0       10  3.0      84  quadratic
13  8.0     10.3  3.0      84  quadratic
     day response dose patient       type
0    5.0      4.6  2.0     114     linear
1    6.0      6.2  2.5     114     linear
2    7.0     10.8  3.0     114     linear
3    8.0      5.8  3.0     114     linear
4    9.0     11.3  3.5     114     linear
5   10.0      7.5  2.0     114     linear
6   11.0      6.8  2.0     114     linear
7   12.0        3  1.0     114     linear
8   13.

In [14]:
cal_pred

,index


In [52]:

result




,patient,method,pred_day,fit_dose_1,fit_dose_2,fit_dose_3,fit_dose_4,fit_dose_5,fit_dose_6,fit_dose_7,...,prev_coeff_2x,prev_coeff_1x,prev_coeff_0x,prev_deviation,coeff_2x,coeff_1x,coeff_0x,prediction,deviation,abs_deviation


In [14]:
a = pd.DataFrame(columns=['col1', 'col2', 'COL1'])
a.loc[0,:] = [1,2,3]
b = pd.DataFrame(columns=['col1', 'col2', 'col3', 'COL1'])
b.loc[0,:] = [1,2,3,4]
pd.concat([a, b])


,col1,col2,COL1,col3
0,1,2,3,NaN
0,1,2,4,3


In [45]:
a = pd.DataFrame(columns=['a', 'b', 'c'])
a.loc[0,'a':'c'] = 5
a.loc[1, 'a': 'c'] = 4
a.loc[0:1-1, 'a']

0    5
Name: a, dtype: object